<a href="https://colab.research.google.com/github/lavanya0479/Agribridge/blob/main/image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install pytorch-lightning torchmetrics pyyaml


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.5/849.5 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 64.4 MB/s eta 0:00:00


In [1]:
%%writefile config.yaml
seed: 42
batch_size: 128
num_workers: 2
max_epochs: 20
learning_rate: 0.001

model:
  num_classes: 10

callbacks:
  early_stopping_patience: 5

logging:
  log_dir: logs


Writing config.yaml


In [2]:
%%writefile model.py
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torchvision import models
from torchmetrics import Accuracy

class CIFAR10Classifier(pl.LightningModule):
    def __init__(self, lr=1e-3, num_classes=10):
        super().__init__()
        self.save_hyperparameters()

        self.model = models.resnet18(pretrained=False)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

        self.train_acc = Accuracy(task="multiclass", num_classes=10)
        self.val_acc = Accuracy(task="multiclass", num_classes=10)
        self.test_acc = Accuracy(task="multiclass", num_classes=10)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        acc = self.train_acc(logits, y)

        self.log("train_loss", loss, prog_bar=True)
        self.log("train_acc", acc, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        acc = self.val_acc(logits, y)

        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        acc = self.test_acc(logits, y)
        self.log("test_acc", acc)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)


Writing model.py


In [3]:
%%writefile train.py
import yaml
import pytorch_lightning as pl
from torchvision import transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

from model import CIFAR10Classifier

# Load config
with open("config.yaml") as f:
    cfg = yaml.safe_load(f)

pl.seed_everything(cfg["seed"])

# Transforms
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Datasets
train_ds = CIFAR10(root="data", train=True, download=True, transform=train_transform)
test_ds = CIFAR10(root="data", train=False, download=True, transform=test_transform)

train_loader = DataLoader(
    train_ds,
    batch_size=cfg["batch_size"],
    shuffle=True,
    num_workers=cfg["num_workers"]
)

test_loader = DataLoader(
    test_ds,
    batch_size=cfg["batch_size"],
    shuffle=False,
    num_workers=cfg["num_workers"]
)

# Model
model = CIFAR10Classifier(
    lr=cfg["learning_rate"],
    num_classes=cfg["model"]["num_classes"]
)

# Callbacks
callbacks = [
    EarlyStopping(
        monitor="val_acc",
        patience=cfg["callbacks"]["early_stopping_patience"],
        mode="max"
    ),
    ModelCheckpoint(
        monitor="val_acc",
        mode="max",
        save_top_k=1
    )
]

# Logger
logger = TensorBoardLogger(
    save_dir=cfg["logging"]["log_dir"],
    name="cifar10"
)

# Trainer
trainer = pl.Trainer(
    max_epochs=cfg["max_epochs"],
    callbacks=callbacks,
    logger=logger,
    accelerator="gpu",   # or "cpu" if GPU not available
    devices=1
)


trainer.fit(model, train_loader, test_loader)
trainer.test(model, test_loader)


Writing train.py


In [7]:
!python train.py


Streaming output truncated to the last 5000 lines.
                                     0:00:08                   train_loss: 0.454
                                                               train_acc: 0.852 
                                                               val_loss: 0.566  
Epoch 17/19 ━━━━━━━━━━━━╺━━━ 297/391 0:00:22 •       13.33it/s v_num: 0.000     
                                     0:00:08                   train_loss: 0.658
                                                               train_acc: 0.789 
                                                               val_loss: 0.566  
Epoch 17/19 ━━━━━━━━━━━━╺━━━ 298/391 0:00:22 •       13.35it/s v_num: 0.000     
                                     0:00:07                   train_loss: 0.595
                                                               train_acc: 0.836 
                                                               val_loss: 0.566  
Epoch 17/19 ━━━━━━━━━━━━╺━━━ 299/391 0:00:22 •       13.35

In [6]:
!pip install tensorboard
